In [1]:
import base64
import time
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
import paho.mqtt.client as mqtt
import serial
import pickle
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.metrics import classification_report
import json

# Serial port parameters
x = serial.Serial('COM4', 9600)  # Change 'COM4' to match your serial port

# Encryption parameters
key = b'\xee\x16\xfb\xdf6|\xe8\xe4n\x0b\xeal\xb4\xf1\x05\xabk\xc0\xe8\x9a\xc2\xe0\xff8\x9e\xd8A\xa7d\xfbN\xf0'
iv = b'\xef?\x82\xb5\xa7\x89\x9b\xeet\xda\x8e\xaa\xc3\xd0{\xe4'
cipher = AES.new(key, AES.MODE_CBC, iv=iv)

key2 = b'\xee\x12\xfb\xdf6|\xe8\xe4n\x0b\xeal\xb4\xf1\x05\xabk\xc0\xe8\x9a\xc2\xe0\xff8\x9e\xd8A\xa7d\xfbN\xf0'
iv2 = b'\xef?\x81\xb5\xa7\x89\x9b\xeet\xda\x8e\xaa\xc3\xd0{\xe4'
cipher2 = AES.new(key2, AES.MODE_CBC, iv =iv2)

data = pd.read_csv("heart_attack_prediction_dataset.csv")
data = data.drop(columns=['Patient ID', 'Blood Pressure', 'Exercise Hours Per Week', 'Sedentary Hours Per Day',  
                          'Triglycerides', 'Physical Activity Days Per Week', 'Sleep Hours Per Day', 'Cholesterol', 
                          'Stress Level', 'Continent', 'Hemisphere', 'Country'])
label_encoder = preprocessing.LabelEncoder()
data['Sex'] = label_encoder.fit_transform(data['Sex'])
data['Diet'] = label_encoder.fit_transform(data['Diet'])
X = data.drop('Heart Attack Risk', axis=1)
y = data['Heart Attack Risk']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

with open("final.pkl", 'rb') as file:  
    model = pickle.load(file)
    
y_predict = model.predict(X_test)

# MQTT parameters
broker_address = "localhost"
port = 1883
mqtt_topic2 = "data"

# check results
print(classification_report(y_test, y_predict)) 
age = -1
sex = -1
diabetes =  -1
family_history =  -1
smoking =  -1
obesity =  -1
alcohol_consumption =  -1
diet = -1
previous_heart_problems = -1
medication_use = -1
income = -1
bmi = -1

def on_message(client2, userdata, message):
    global age, sex, diabetes, family_history, smoking, obesity, alcohol_consumption, diet, previous_heart_problems, medication_use, income, bmi
    try:
        # Decode base64 and decrypt the received ciphertext
        ciphertext = base64.b64decode(message.payload)
        decrypted_data = unpad(cipher2.decrypt(ciphertext), AES.block_size)
        
        decoded_data = json.loads(decrypted_data.decode('utf-8').strip())  # Adjust the decoding based on your data format
        
        print(f"Received decrypted data: {decoded_data}")
        age = decoded_data["age"]
        sex = decoded_data["sex"]
        diabetes =  decoded_data["diabetes"]
        family_history =  decoded_data["family_history"]
        smoking =  decoded_data["smoking"]
        obesity =  decoded_data["obesity"]
        alcohol_consumption =  decoded_data["alcohol_consumption"]
        diet = decoded_data["diet"]
        previous_heart_problems = decoded_data["previous_heart_problems"]
        medication_use = decoded_data["medication_use"]
        income = decoded_data["income"]
        bmi = decoded_data["bmi"]
        client2.loop_stop()
    except Exception as e:
        print(f"Error processing message: {e}")

client2 = mqtt.Client("sub")
client2.on_message = on_message

try:
    # Connect to the MQTT Broker
    client2.connect(broker_address, port)
    print("Connected to MQTT broker")
    
    # Subscribe to the MQTT topic
    client2.subscribe(mqtt_topic2)
    print(f"Subscribed to topic: {mqtt_topic2}")
    
    # Start the MQTT loop to listen for messages
    client2.loop_start()  # Use loop_forever instead of disconnect()
except Exception as e:
    print(f"Error: {e}")


mqtt_topic = "bpm/data"
client = mqtt.Client("Publisher")
client.connect(broker_address, port)
client.loop_start()
accumulated_data = []
start_time = time.time()

try:
    while x.isOpen():
        # Read data from the serial port
        data = str(x.readline(), 'utf-8').strip()  # Read and strip any leading/trailing whitespace
        print(data)
        parsed_data = [int(val) for val in data.split(',')]
        accumulated_data.append(parsed_data)
        
        if time.time() - start_time >= 10:
            # Calculate the average of accumulated data
            averaged_data = sum(sum(sublist) for sublist in accumulated_data) / len(accumulated_data)
            print(type(averaged_data))
            test = [[int(age), int(sex), int(averaged_data), int(diabetes), int(family_history), int(smoking), int(obesity), int(alcohol_consumption), int(diet), int(previous_heart_problems), int(medication_use), int(income), int(bmi)]]
            print(test)
            y_predict = model.predict(test)
            print(y_predict)
            y_predict_str = ','.join(map(str, y_predict))
            
            # Serialize the data to JSON format
            transfer_data = json.dumps({"y_predict": y_predict_str, "sensor_data": averaged_data})
            
            # Encrypt the serialized data
            sensor_data = pad(transfer_data.encode('utf-8'), AES.block_size)  # Ensure the data is padded to match AES block size
            ciphertext_data = cipher.encrypt(sensor_data)
            
            # Publish the encrypted data to MQTT
            client.publish(mqtt_topic, base64.b64encode(ciphertext_data))
            
            print(f"Original data: {averaged_data}")
            print(f"Encrypted data: {ciphertext_data}")
            # Reset variables for next iteration
            accumulated_data = []
            start_time = time.time()
            
        time.sleep(1)  # Adjust the delay as needed
finally:
    # Close the serial port and disconnect from MQTT
    x.close()
    client.loop_stop()
    client.disconnect()



c:\Users\LENOVO\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\LENOVO\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\LENOVO\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera

              precision    recall  f1-score   support

           0       0.64      1.00      0.78      1691
           1       0.00      0.00      0.00       938

    accuracy                           0.64      2629
   macro avg       0.32      0.50      0.39      2629
weighted avg       0.41      0.64      0.50      2629

Connected to MQTT broker
Subscribed to topic: data
Received decrypted data: {'age': 22, 'sex': 0, 'diabetes': 0, 'family_history': 0, 'smoking': 0, 'obesity': 0, 'alcohol_consumption': 0, 'diet': 0, 'previous_heart_problems': 0, 'medication_use': 0, 'income': 21000, 'bmi': 22}
24
<class 'float'>
[[22, 0, 24, 0, 0, 0, 0, 0, 0, 0, 0, 21000, 22]]
[0]
Original data: 24
Encrypted data: b"\xc5Q\xd8\xad\x9cQ\xaa\xf2\xaf\xdf\xdd6r}\xa03q\xee[\xdc_\xe7&'\x01u\xb0\x02\x13\x95\x95s\xd1^l\xed\xf7\xc5P%\xf5\xa73\r\x0e(\x1d)"


c:\Users\LENOVO\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


37
65
<class 'float'>
[[22, 0, 51, 0, 0, 0, 0, 0, 0, 0, 0, 21000, 22]]
[0]
Original data: 65
Encrypted data: b'\xaeN\x9bw\xff\x17\xbe\xf6,\xa4\xb8{U<\x13;\xf6\xe9\x1f\xf5/\xf2\xbf\xb6\xa9\xe7Ov8V\x8f\x9a\xce\xc0\xea\xa0\x1d\x1b]\xabN}b\xdb=\xee\xfbJ'


c:\Users\LENOVO\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


...

c:\Users\LENOVO\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


147
166
183
<class 'float'>
[[22, 0, 165, 0, 0, 0, 0, 0, 0, 0, 0, 21000, 22]]
[0]
Original data: 183
Encrypted data: b'\xbaK\xdcS\xfe\x1d\x12l\xf8\xcd\xeb \xfa\xf1\r}\xa3\xa6\x86\xc0\xe1*\xc0S&\xcc\xb2\xde\x0b\x91b\xe4"F\xc5n\rr\x90\x88\x03k\x16\x96\x04M\xb3a'


c:\Users\LENOVO\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


151
204
167
<class 'float'>
[[22, 0, 174, 0, 0, 0, 0, 0, 0, 0, 0, 21000, 22]]
[0]
Original data: 167
Encrypted data: b">y\x92\x82L)\xff\x10\x97\xe0\x86U\xff\xe9\xb1\x97\xa52_\xc3`\xbc]k\x99\xf4\xdb\xb0\xfa'\x91\xae\xd9c\x18\x96$h]\x9a\xea\x8b\x87\xcajF\xea\n"


c:\Users\LENOVO\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
